In [ ]:
import sys

sys.path.append("../..")
sys.path.append("../../../py-modelrunner")
sys.path.append("../../../py-pde")

import time
from collections import Counter
from itertools import product

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd

from tqdm.auto import tqdm

In [ ]:
import multicomp as mm

# Test Individual

In [ ]:
ind = mm.FullChiIndividual({"num_comp_init": 4, "repetitions": 3})
# ind.show_parameters()

In [ ]:
res = ind.get_ensemble("phase_counts")
res

In [ ]:
ind.diagnostics

# Test Population

In [ ]:
p_ind = {
    "num_comp_init": 5,
    "repetitions": 3,
    "mutation_size": 1.0,
}

In [ ]:
pop = mm.Population(
    [mm.FullChiIndividual(p_ind) for _ in range(4)], {"num_processes": 4}
)

In [ ]:
pop.show_parameters()

In [ ]:
stats = pop.get_stats(["phase_counts", "phis"])

In [ ]:
stat = stats[0]
stat["phis"]

# Environment

In [ ]:
env = mm.TargetPhaseCountEnvironment({"target_phase_count": 3})

In [ ]:
stats = pop.get_stats(["phase_counts"])

In [ ]:
# determine fitness of each individual
target_count = env.parameters["target_phase_count"]
fitness_tol = env.parameters["phase_count_tolerance"]
fitnesses = np.empty(len(pop))
for i, stat in enumerate(stats):
    counts = np.asarray(stat["phase_counts"])
    arg = (counts - target_count) / fitness_tol
    fitnesses[i] = np.mean(np.exp(-0.5 * arg**2))
fitnesses

In [ ]:
env.evolve(pop)

In [ ]:
pop = mm.Population([mm.FullChiIndividual(), mm.FullChiIndividual()])
env = mm.PartitioningEnvironment({"enriched_components": [0]})
env.evolve(pop)

In [ ]:
np.array(env.parameters["enriched_components"])

In [ ]:
env.get_population_fitness(pop)

# Test

In [ ]:
# determine the statistics of the population
stats = pop.get_stats(["phase_counts", "phis"])

In [ ]:
# read parameters
target_count = env.parameters["target_phase_count"]
phase_count_tol = env.parameters["phase_count_tolerance"]
enriched_components = np.array(env.parameters["enriched_components"])
weight = env.parameters["fitness_weight"]

In [ ]:
# determine fitness of each individual
fitnesses = np.empty(len(pop))
for i, ind_stat in enumerate(stats):  # iterate over individuals
    # determine phase count fitness
    counts = np.asarray(ind_stat["phase_counts"])
    arg = (counts - target_count) / phase_count_tol
    fitness_count = np.mean(np.exp(-0.5 * arg**2))

    if len(enriched_components) > 0:
        # determine maximal partition coefficient
        result = []
        for phis in ind_stat["phis"]:  # iterate over ensemble
            # phis is now a 2d array of compositions (phases x component)
            phis_sel = phis[:, enriched_components]
            K = len(phis_sel)  # phase count of the individual sample
            if K == 1:
                result.append(np.array([0]))
            else:
                partition = (K * phis_sel - phis_sel.sum(axis=0)) / (K - 1)
                result.append(np.max(partition, axis=0))

        # mean best partitioning
        fitness_partition = np.mean(result, axis=0)
        print(fitness_count, fitness_partition, result)
        fitnesses[i] = weight * fitness_count + (1 - weight) * fitness_partition

    else:
        fitnesses[i] = fitness_count
fitnesses